In [39]:
import pandas as pd

# Define the file path for the .dta file
filepath = r'C:\Users\azraj\OneDrive\Desktop\MA Economics\Fall Term\Causal ML\Project\Targetted Ads\replication_package_MS-INS-21-00828\at.dta'  # Update this path to the location of your .dta file

# Load the .dta file
panel_data = pd.read_stata(filepath)
print(panel_data.columns)

Index(['AFTER', 'AFTERXCharges_Price', 'AFTERXTop_Demanded', 'AFTERXTop_Rated',
       'AFTERXUndiversified', 'AFTERXYoung', 'Ad_Networks', 'Ad_Views', 'Age',
       'Bug_Fix', 'Charges_Price', 'Collects_User_ID', 'Demand',
       'Feature_Update', 'Feature_Update_lag', 'Feature_Updates', 'File_Size',
       'Firm_Size', 'Log_Demand', 'Log_Firm_Size', 'Log_Price', 'One_Employee',
       'Permissions', 'Price', 'Rating', 'TREATXAFTER',
       'TREATXAFTERXCharges_Price', 'TREATXAFTERXTop_Demanded',
       'TREATXAFTERXTop_Rated', 'TREATXAFTERXUndiversified',
       'TREATXAFTERXYoung', 'Top_Demanded', 'Top_Rated', 'Undiversified',
       'Update', 'Young', 'ad_networks_above_median', 'ads_above_median',
       'ann_TREAT', 'app_num', 'collected_id', 'genre_cons', 'genre_num',
       'ym'],
      dtype='object')


In [31]:
def prepare_data(panel_data):
    """Prepare data for regression analysis"""
    df = panel_data.copy()
    
    # Convert to panel format
    df = df.set_index(['app_num', 'ym'])
    
    # Ensure datatypes
    numeric_cols = ['Feature_Update', 'TREATXAFTER', 'Log_Demand', 'Age', 
                    'Price', 'Log_Firm_Size', 'One_Employee', 'Feature_Update_lag',
                    'Rating', 'File_Size']
    for col in numeric_cols:
        df[col] = df[col].astype(float)
    
    df['genre_num'] = df['genre_num'].astype('category')
    
    return df

def run_model(data, dependent_var, controls=True):
    """Run the DiD model for the specified dependent variable"""
    exog_vars = ['TREATXAFTER']
    
    # Add controls
    if controls:
        additional_vars = ['Age', 'Price', 'Log_Firm_Size', 'One_Employee', 'Feature]
        if dependent_var != 'Log_Demand':
            additional_vars.append('Log_Demand')
        exog_vars.extend(additional_vars)
    
    # Drop rows with NaN in the dependent variable
    data = data.dropna(subset=[dependent_var])
    
    # Create the model
    model = PanelOLS(
        dependent=data[dependent_var],
        exog=data[exog_vars],
        entity_effects=True,
        time_effects=True,
        check_rank=False
    )
    
    # Fit with clustered standard errors
    results = model.fit(cov_type='clustered', cluster_entity=True)
    
    return results

def print_formatted_results(results, dependent_var):
    """Print formatted results"""
    print(f"\nResults for {dependent_var}:")
    print("-" * 50)
    
    # Print coefficients and standard errors for all variables
    for var in results.params.index:
        coef = results.params[var]
        se = results.std_errors[var]
        stars = ''
        pval = results.pvalues[var]
        if pval < 0.01:
            stars = '***'
        elif pval < 0.05:
            stars = '**'
        elif pval < 0.1:
            stars = '*'
        print(f"{var}: {coef:.4f}{stars}")
        print(f"Standard Error: ({se:.4f})")
    
    print(f"\nObservations: {results.nobs:,}")
    print(f"Game fixed effects: x")
    print(f"Month fixed effects: x")
    print(f"R-squared: {results.rsquared:.4f}")

def run_new_models(data):
    """Run models for Rating, Log_Demand, and File_Size"""
    dependent_vars = ['Rating', 'Log_Demand', 'File_Size']
    results = {}
    
    for dv in dependent_vars:
        print(f"\n{'='*60}")
        print(f"Running model for {dv}")
        print(f"{'='*60}")
        
        model_results = run_model(data, dv)
        print_formatted_results(model_results, dv)
        results[dv] = model_results
    
    return results

def main():
    # Load the data
    filepath = r'C:\Users\azraj\OneDrive\Desktop\MA Economics\Fall Term\Causal ML\Project\Targetted Ads\replication_package_MS-INS-21-00828\at.dta'
    panel_data = pd.read_stata(filepath)
    
    # Prepare the data
    panel_data = prepare_data(panel_data)
    
    # Run new models
    new_results = run_new_models(panel_data)
    
    return new_results, panel_data

if __name__ == "__main__":
    results, panel_data = main()


Running model for Rating


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for Rating:
--------------------------------------------------
TREATXAFTER: 0.0176**
Standard Error: (0.0078)
Age: -0.0000
Standard Error: (0.0001)
Price: -0.0257
Standard Error: (0.0292)
Log_Firm_Size: -0.0050
Standard Error: (0.0072)
One_Employee: -0.0365*
Standard Error: (0.0214)
Feature_Update: 0.0123**
Standard Error: (0.0048)
Log_Demand: -0.0769***
Standard Error: (0.0078)

Observations: 88,566
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0246

Running model for Log_Demand


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for Log_Demand:
--------------------------------------------------
TREATXAFTER: -0.0256
Standard Error: (0.0178)
Age: -0.0003*
Standard Error: (0.0002)
Price: -1.1874**
Standard Error: (0.4895)
Log_Firm_Size: 0.1755***
Standard Error: (0.0220)
One_Employee: 0.0047
Standard Error: (0.0589)
Feature_Update: -0.0786***
Standard Error: (0.0249)

Observations: 93,881
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0110

Running model for File_Size


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for File_Size:
--------------------------------------------------
TREATXAFTER: -0.0822
Standard Error: (0.1780)
Age: -0.0024
Standard Error: (0.0016)
Price: -5.5422
Standard Error: (4.4768)
Log_Firm_Size: -0.1575
Standard Error: (0.2130)
One_Employee: -1.0756
Standard Error: (0.9424)
Feature_Update: 1.2139***
Standard Error: (0.3387)
Log_Demand: 0.3238**
Standard Error: (0.1362)

Observations: 73,067
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0031


In [35]:
def prepare_data(panel_data):
    """Prepare data for regression analysis"""
    df = panel_data.copy()
    
    # Convert to panel format
    df = df.set_index(['app_num', 'ym'])
    
    # Ensure datatypes
    numeric_cols = ['Rating', 'TREATXAFTER', 'Log_Demand', 'Age', 'Price', 'Log_Firm_Size', 'One_Employee']
    for col in numeric_cols:
        df[col] = df[col].astype(float)
    
    df['genre_num'] = df['genre_num'].astype('category')
    
    # Drop rows with NaN in the dependent variable (Rating)
    df = df.dropna(subset=['Rating'])
    
    return df

def run_ratings_model(data, additional_vars=None):
    """Run the DiD model for the Ratings dependent variable"""
    exog_vars = ['TREATXAFTER']
    
    # Add controls
    if additional_vars:
        exog_vars.extend(additional_vars)
    
    # Create the model
    model = PanelOLS(
        dependent=data['Rating'],
        exog=data[exog_vars],
        entity_effects=True,
        time_effects=True,
        check_rank=False
    )
    
    # Fit with clustered standard errors
    results = model.fit(cov_type='clustered', cluster_entity=True)
    
    return results

def print_formatted_results(results):
    """Print formatted results for the Ratings model"""
    print("\nResults for Rating:")
    print("-" * 50)
    
    # Print coefficients and standard errors for all variables
    for var in results.params.index:
        coef = results.params[var]
        se = results.std_errors[var]
        stars = ''
        pval = results.pvalues[var]
        if pval < 0.01:
            stars = '***'
        elif pval < 0.05:
            stars = '**'
        elif pval < 0.1:
            stars = '*'
        print(f"{var}: {coef:.4f}{stars}")
        print(f"Standard Error: ({se:.4f})")
    
    print(f"\nObservations: {results.nobs:,}")
    print("Game fixed effects: x")
    print("Month fixed effects: x")
    print(f"R-squared: {results.rsquared:.4f}")

def main():
    # Load the data
    filepath = r'C:\Users\azraj\OneDrive\Desktop\MA Economics\Fall Term\Causal ML\Project\Targetted Ads\replication_package_MS-INS-21-00828\at.dta'
    panel_data = pd.read_stata(filepath)
    
    # Prepare the data
    panel_data = prepare_data(panel_data)
    
    # Run the Ratings model
    base_model_results = run_ratings_model(panel_data)
    print_formatted_results(base_model_results)
    
    # Run the Ratings model with additional variables
    additional_vars = ['Log_Demand', 'Feature_Update']
    extended_model_results = run_ratings_model(panel_data, additional_vars)
    print_formatted_results(extended_model_results)
    
    return base_model_results, extended_model_results, panel_data

if __name__ == "__main__":
    base_results, extended_results, panel_data = main()


Results for Rating:
--------------------------------------------------
TREATXAFTER: 0.0221***
Standard Error: (0.0083)

Observations: 93,428
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0006


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for Rating:
--------------------------------------------------
TREATXAFTER: 0.0175**
Standard Error: (0.0078)
Log_Demand: -0.0771***
Standard Error: (0.0078)
Feature_Update: 0.0124***
Standard Error: (0.0048)

Observations: 88,566
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0244


In [ ]:
from sklearn import tree
sqft_tree = tree.DecisionTreeRegressor(max_depth=3).fit(X,Y)
# use the fitted tree to predict
y_pred_tree = sqft_tree.predict(X)

# find the error of prediction (MSE)
from sklearn import metrics
print('Mean Squared Error:', metrics.mean_squared_error(Y, y_pred_tree))
sqrf_fig = plt.figure(figsize=(25,20))
sqrf_fig = tree.plot_tree(sqft_tree, feature_names=X.columns, filled=True)

In [ ]:
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
clf = DTC(criterion='entropy',
          max_depth=3,
          random_state=0)        
clf.fit(X, y)

ax = subplots(figsize=(12,12))[1]
feature_names = X.columns
plot_tree(clf,
          feature_names=feature_names,
          ax=ax);


In [ ]:
#Random Forest: using 5 features

# define and fit
regr_clf = RandomForestClassifier(max_features=5, random_state=1).fit(X, y)

#predict
pred = regr_clf.predict(X)

#calculate MSE
rf_mse = mean_squared_error(y, pred)
Importance = pd.DataFrame({'Importance':regr_clf.feature_importances_*100}, index=X.columns)
Importance.sort_values('Importance', axis=0, ascending=True).plot(kind='barh', color='r', )
plt.title('Importance Score Plot')
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_predict

# Convert the data into XGBoost's DMatrix format
dtrain = xgb.DMatrix(X, label=y)

# Define the parameters for the XGBoost model
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Train the XGBoost model with the optimal number of boosting rounds
model = xgb.train(params, dtrain, num_boost_round= 10)

# Make predictions 
y_pred = model.predict(dtrain)

# Calculate and print the Mean Squared Error (MSE)
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)


plt.figure(figsize=(8, 6))
plt.scatter(y, y_pred, c='grey', alpha=0.3)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
plt.xlabel('True Y')
plt.ylabel('Predicted Y')
plt.title('Y vs Predicted Y (Y hat)')
plt.show()

In [ ]:
# Define cross-validation strategy
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize LassoCV with cross-validation and a custom range of alphas
custom_alphas = np.logspace(-5, 1, 100)  # Custom range of alphas
lassoCV = LassoCV(
    alphas=custom_alphas,  # Use the custom alphas
    cv=kfold,
    max_iter=100000,
    tol=1e-4,
    random_state=42
)

# Create a pipeline with scaler and LassoCV
pipeCV = Pipeline(steps=[('scaler', scaler),
                         ('lasso', lassoCV)])

# Fit the pipeline to the data
pipeCV.fit(X, Y)
print("\nLassoCV model fitted successfully.")

# Retrieve the tuned alpha (regularization parameter)
tuned_lasso = pipeCV.named_steps['lasso']
best_alpha = tuned_lasso.alpha_
print(f"\nOptimal alpha determined by cross-validation: {best_alpha}")

# ------------------------------------------------------------
# Plotting Cross-validated MSE with Error Bars
# ------------------------------------------------------------

# Extract the cross-validated MSEs and standard deviations
mse_path_mean = np.mean(tuned_lasso.mse_path_, axis=1)
mse_path_std = np.std(tuned_lasso.mse_path_, axis=1)

# Plotting the cross-validated MSE vs. -log(lambda) with error bars
plt.figure(figsize=(8, 8))
plt.errorbar(-np.log10(tuned_lasso.alphas_), mse_path_mean, yerr=mse_path_std / np.sqrt(5), fmt='o-')
plt.axvline(-np.log10(best_alpha), color='k', linestyle='--')
plt.xlabel('$-\\log(\\lambda)$', fontsize=20)
plt.ylabel('Cross-validated MSE', fontsize=20)
plt.title('Lasso Regression Cross-Validated MSE', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Plotting Lasso Coefficients Path
# ------------------------------------------------------------

# Compute the Lasso path (coefficients for different alphas)
X_scaled = scaler.transform(X)  # Lasso expects standardized data

# Define a range of alpha values (logarithmically spaced)
num_alphas = 100
alphas_lasso = np.logspace(-5, 1, num_alphas)

# Compute the Lasso path
alphas_lasso, coefs_lasso, _ = lasso_path(X_scaled, Y, alphas=alphas_lasso)

# Create a DataFrame for the solution path
feature_names = X.columns
soln_path = pd.DataFrame(coefs_lasso.T, columns=feature_names, index=-np.log10(alphas_lasso))

# Plot the Lasso coefficient path
plt.figure(figsize=(10, 6))
for feature in feature_names:
    plt.plot(soln_path.index, soln_path[feature], label=feature)

plt.xlabel('$-\\log(\\lambda)$', fontsize=14)
plt.ylabel('Standardized Coefficients', fontsize=14)
plt.title('Lasso Regression Plot', fontsize=16)
plt.legend(title='Features')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define a range of lambda values (regularization strengths)
lambda_min_exponent = -5
lambda_max_exponent = 8
num_lambdas = 100
lambdas = 10 ** np.linspace(lambda_max_exponent, lambda_min_exponent, num_lambdas)

# Fit Elastic Net model paths with l1_ratio=0 (Ridge Regression)
alphas, coefs, _ = enet_path(X_scaled, Y, l1_ratio=0.0, alphas=lambdas)

# Create a DataFrame for the solution path
coef_df = pd.DataFrame(coefs, index=['TREATXAFTER', 'Age', 'Log_Firm_Size', 'Price', 'One_Employee'], columns=-np.log(alphas))

# Plot the solution path
plt.figure(figsize=(10, 6))
for feature in coef_df.index:
    plt.plot(coef_df.columns, coef_df.loc[feature], label=feature)

plt.xlabel('$-\log(\lambda)$', fontsize=14)
plt.ylabel('Standardized Coefficients', fontsize=14)
plt.title('Ridge Regression Plot', fontsize=16)
plt.legend(title='Features')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define a range of lambda values (regularization strengths)
lambda_min_exponent = -8
lambda_max_exponent = 8
num_lambdas = 100
lambdas = 10 ** np.linspace(lambda_max_exponent, lambda_min_exponent, num_lambdas)

# Set up the pipeline with scaling and Ridge regression
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

# Set up the parameter grid for alpha (lambda)
param_grid = {'ridge__alpha': lambdas}

# Use GridSearchCV to perform cross-validation over the lambdas
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X, Y)

# Get the cross-validated MSEs and standard deviations
mean_test_scores = -grid_search.cv_results_['mean_test_score']
std_test_scores = grid_search.cv_results_['std_test_score']

# Get the best alpha (lambda) value
best_alpha = grid_search.best_estimator_.named_steps['ridge'].alpha

# Plotting the cross-validated MSE vs. -log(lambda)
plt.figure(figsize=(8, 8))
plt.errorbar(-np.log(lambdas), mean_test_scores, yerr=std_test_scores / np.sqrt(5), fmt='o-')
plt.axvline(-np.log(best_alpha), color='k', linestyle='--')
plt.xlabel('$-\\log(\\lambda)$', fontsize=20)
plt.ylabel('Cross-validated MSE', fontsize=20)
plt.title('Ridge Regression Cross-Validated MSE', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()

# Get and print the best alpha (lambda) value
best_alpha = grid_search.best_estimator_.named_steps['ridge'].alpha
print(f"Optimal alpha (lambda) value: {best_alpha}")



In [ ]:
#Bagging; using all features

regr_bagg = RandomForestClassifier(max_features= 18, random_state=1) 
regr_bagg.fit(X, y)


pred = regr_bagg.predict(X)

plt.scatter(pred, y, label='log price')
plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
plt.xlabel('pred')
plt.ylabel('y')


bag_mse = mean_squared_error(y, pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
import sklearn.model_selection as skm
(X_train,
 X_test,
 y_train,
 y_test) = skm.train_test_split(X,
                                y,
                                test_size=0.3,
                                random_state=0)
boost_boston = GBC(n_estimators=5000,
                   learning_rate=0.001,
                   max_depth=3,
                   random_state=0)
boost_boston.fit(X_train, y_train)
test_error = np.zeros_like(boost_boston.train_score_)
for idx, y_ in enumerate(boost_boston.staged_predict(X_test)):
   test_error[idx] = np.mean((y_test - y_)**2)

plot_idx = np.arange(boost_boston.train_score_.shape[0])
ax = subplots(figsize=(8,8))[1]
ax.plot(plot_idx,
        boost_boston.train_score_,
        'b',
        label='Training')
ax.plot(plot_idx,
        test_error,
        'r',
        label='Test')
ax.legend();
y_hat_boost = boost_boston.predict(X_test);
boost_mse = np.mean((y_test - y_hat_boost)**2)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Assuming X and y are already defined

# Encode target labels if they are categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dictionary to store models and results
models = {
    'Random Forest': RandomForestClassifier(max_features=5, random_state=0),
    'Bagging': BaggingClassifier(random_state=0),
    'Boosting': GradientBoostingClassifier(n_estimators=5000, learning_rate=0.001, max_depth=3,random_state=0),
    'Decision Tree': DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
}

# Loop through each model, fit, predict and calculate MSE and RMSE
results = []
for model_name, model in models.items():
    # Train the model
    model.fit(X, y_encoded)
    
    # Predict class probabilities
    y_prob = model.predict_proba(X)
    
    # Get the predicted class
    y_pred = np.argmax(y_prob, axis=1)
    
    # Calculate MSE
    mse = mean_squared_error(y_encoded, y_pred)
    
    # Calculate RMSE
    rmse = np.sqrt(mse)
    
    # Store the results
    results.append({
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse
    })

# Convert results to a DataFrame and display
import pandas as pd
df_results = pd.DataFrame(results)
print(df_results)



In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import econml
from dowhy import CausalModel
from sklearn.linear_model import LassoCV

estimate_dml = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.DML',
    method_params={
        'init_params': {
            'model_y': GradientBoostingRegressor(),
            'model_t': GradientBoostingRegressor(),
            'model_final': LassoCV(fit_intercept=False),
        },
        'fit_params': {}}
)

print(f'Estimate of causal effect (DML): {estimate_dml.value}')

In [ ]:
estimate_lr = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.linear_regression')

print(f'Estimate of causal effect (linear regression): {estimate_lr.value}')

In [ ]:
random_cause = model.refute_estimate(
    estimand=estimand, 
    estimate=estimate,
    method_name='random_common_cause'
)
print(random_cause)

In [ ]:
placebo_refuter = model.refute_estimate(
    estimand=estimand, 
    estimate=estimate,
    method_name='placebo_treatment_refuter'
)
print(placebo_refuter)

In [105]:
import pandas as pd
from linearmodels import PanelOLS

def prepare_data(panel_data):
    """Prepare data for regression analysis"""
    df = panel_data.copy()
    
    # Convert to panel format
    df = df.set_index(['app_num', 'ym'])
    
    # Ensure datatypes
    numeric_cols = ['Feature_Update', 'TREATXAFTER', 'Log_Demand', 'Age', 
                    'Price', 'Log_Firm_Size', 'One_Employee', 'Feature_Update_lag',
                    'Rating', 'File_Size']
    for col in numeric_cols:
        df[col] = df[col].astype(float)
    
    df['genre_num'] = df['genre_num'].astype('category')
    
    return df

def run_model(data, dependent_var, controls=True):
    """Run the DiD model for the specified dependent variable"""
    exog_vars = ['TREATXAFTER']
    
    # Add controls
    if controls:
        additional_vars = ['Age', 'Price', 'Log_Firm_Size', 'One_Employee', 'Feature_Update']
        if dependent_var != 'Log_Demand':
            additional_vars.append('Log_Demand')
        exog_vars.extend(additional_vars)
    
    # Drop rows with NaN in the dependent variable
    data = data.dropna(subset=[dependent_var])
    
    # Create the model
    model = PanelOLS(
        dependent=data[dependent_var],
        exog=data[exog_vars],
        entity_effects=True,
        time_effects=True,
        check_rank=False
    )
    
    # Fit with clustered standard errors
    results = model.fit(cov_type='clustered', cluster_entity=True)
    
    return results

def print_formatted_results(results, dependent_var):
    """Print formatted results"""
    print(f"\nResults for {dependent_var}:")
    print("-" * 50)
    
    # Print coefficients and standard errors for all variables
    for var in results.params.index:
        coef = results.params[var]
        se = results.std_errors[var]
        stars = ''
        pval = results.pvalues[var]
        if pval < 0.01:
            stars = '***'
        elif pval < 0.05:
            stars = '**'
        elif pval < 0.1:
            stars = '*'
        print(f"{var}: {coef:.4f}{stars}")
        print(f"Standard Error: ({se:.4f})")
    
    print(f"\nObservations: {results.nobs:,}")
    print(f"Game fixed effects: x")
    print(f"Month fixed effects: x")
    print(f"R-squared: {results.rsquared:.4f}")

def run_new_models(data):
    """Run models for Rating, Log_Demand, and File_Size"""
    dependent_vars = ['Rating', 'Log_Demand', 'File_Size']
    results = {}
    
    for dv in dependent_vars:
        print(f"\n{'='*60}")
        print(f"Running model for {dv}")
        print(f"{'='*60}")
        
        model_results = run_model(data, dv)
        print_formatted_results(model_results, dv)
        results[dv] = model_results
    
    return results

def main():
    # Load the data
    filepath = r'C:\Users\azraj\OneDrive\Desktop\MA Economics\Fall Term\Causal ML\Project\Targetted Ads\replication_package_MS-INS-21-00828\at.dta'
    panel_data = pd.read_stata(filepath)
    
    # Prepare the data
    panel_data = prepare_data(panel_data)
    
    # Run new models
    new_results = run_new_models(panel_data)
    
    # Save results to a CSV file
    results_df = pd.DataFrame()
    for dv, model_results in new_results.items():
        for var in model_results.params.index:
            row = {
                'Dependent_Variable': dv,
                'Independent_Variable': var,
                'Coefficient': model_results.params[var],
                'Standard_Error': model_results.std_errors[var],
                'P_Value': model_results.pvalues[var],
                'R_Squared': model_results.rsquared
            }
            results_df = results_df.append(row, ignore_index=True)
    
    results_df.to_csv('regression_results.csv', index=False)
    
    return new_results, panel_data

if __name__ == "__main__":
    results, panel_data = main()


Running model for Rating


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for Rating:
--------------------------------------------------
TREATXAFTER: 0.0176**
Standard Error: (0.0078)
Age: -0.0000
Standard Error: (0.0001)
Price: -0.0257
Standard Error: (0.0292)
Log_Firm_Size: -0.0050
Standard Error: (0.0072)
One_Employee: -0.0365*
Standard Error: (0.0214)
Feature_Update: 0.0123**
Standard Error: (0.0048)
Log_Demand: -0.0769***
Standard Error: (0.0078)

Observations: 88,566
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0246

Running model for Log_Demand


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for Log_Demand:
--------------------------------------------------
TREATXAFTER: -0.0256
Standard Error: (0.0178)
Age: -0.0003*
Standard Error: (0.0002)
Price: -1.1874**
Standard Error: (0.4895)
Log_Firm_Size: 0.1755***
Standard Error: (0.0220)
One_Employee: 0.0047
Standard Error: (0.0589)
Feature_Update: -0.0786***
Standard Error: (0.0249)

Observations: 93,881
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0110

Running model for File_Size


C:\Users\azraj\anaconda3\Lib\site-packages\linearmodels\panel\model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)



Results for File_Size:
--------------------------------------------------
TREATXAFTER: -0.0822
Standard Error: (0.1780)
Age: -0.0024
Standard Error: (0.0016)
Price: -5.5422
Standard Error: (4.4768)
Log_Firm_Size: -0.1575
Standard Error: (0.2130)
One_Employee: -1.0756
Standard Error: (0.9424)
Feature_Update: 1.2139***
Standard Error: (0.3387)
Log_Demand: 0.3238**
Standard Error: (0.1362)

Observations: 73,067
Game fixed effects: x
Month fixed effects: x
R-squared: 0.0031


AttributeError: 'DataFrame' object has no attribute 'append'

In [121]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, lasso_path, enet_path, Ridge
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

def prepare_data(panel_data):
    """Prepare data for regression analysis"""
    df = panel_data.copy()
    
    # Convert to panel format
    df = df.set_index(['app_num', 'ym'])
    
    # Ensure datatypes
    numeric_cols = [
        'Feature_Update', 'Log_Demand', 'Age', 'Price', 'Log_Firm_Size',
        'One_Employee', 'Feature_Update_lag', 'Rating', 'File_Size'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    df['genre_num'] = df['genre_num'].astype('category')
    
    return df

def run_regularized_models(data):
    """Run LASSO, Ridge, and Elastic Net models"""
    indep_vars = [
        'Ad_Networks', 'Ad_Views', 'Age', 'Bug_Fix', 'Charges_Price',
        'Collects_User_ID', 'Feature_Update', 'File_Size', 'Firm_Size', 'Log_Demand',
        'Log_Firm_Size', 'Log_Price', 'One_Employee', 'Permissions',
        'Price', 'Top_Demanded', 'Top_Rated', 'Undiversified',
        'Update', 'Young', 'ad_networks_above_median', 'ads_above_median',
        'ann_TREAT', 'app_num', 'collected_id', 'genre_cons', 'genre_num'
    ]

    # Ensure all independent variables are present in the data
    indep_vars = [var for var in indep_vars if var in data.columns]

    # Separate numeric and categorical variables
    numeric_vars = data[indep_vars].select_dtypes(include=['number']).columns.tolist()
    categorical_vars = data[indep_vars].select_dtypes(include=['category', 'object']).columns.tolist()

    # Fill missing values in numeric variables
    X_numeric = data[numeric_vars].fillna(0)

    # Fill missing values in categorical variables
    X_categorical = data[categorical_vars].fillna('missing')

    # Encode categorical variables using one-hot encoding
    X_categorical_encoded = pd.get_dummies(X_categorical, drop_first=True)

    # Combine numeric and encoded categorical variables
    X = pd.concat([X_numeric, X_categorical_encoded], axis=1)

    # Prepare the target variable
    Y = data['Rating'].fillna(data['Rating'].mean())

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Define cross-validation strategy
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # LASSO Regression
    lasso_cv = LassoCV(cv=kfold, random_state=42, max_iter=10000)
    lasso_cv.fit(X_scaled, Y)
    best_alpha_lasso = lasso_cv.alpha_
    print(f"\nOptimal alpha for LASSO: {best_alpha_lasso}")
    
    # Compute LASSO path
    alphas_lasso, coefs_lasso, _ = lasso_path(X_scaled, Y, alphas=None)
    plot_coefficient_convergence(alphas_lasso, coefs_lasso, X.columns, "LASSO Regression Coefficients")
    
    # Ridge Regression
    ridge_cv = RidgeCV(cv=kfold, alphas=np.logspace(-5, 5, 100))
    ridge_cv.fit(X_scaled, Y)
    best_alpha_ridge = ridge_cv.alpha_
    print(f"\nOptimal alpha for Ridge: {best_alpha_ridge}")
    
    # Compute Ridge path
    alphas_ridge = np.logspace(-5, 5, 100)
    coefs_ridge = []
    for alpha in alphas_ridge:
        ridge = Ridge(alpha=alpha, fit_intercept=True)
        ridge.fit(X_scaled, Y)
        coefs_ridge.append(ridge.coef_)
    coefs_ridge = np.array(coefs_ridge).T
    plot_coefficient_convergence(alphas_ridge, coefs_ridge, X.columns, "Ridge Regression Coefficients")
    
    # Elastic Net Regression
    elastic_net_cv = ElasticNetCV(cv=kfold, random_state=42, max_iter=10000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1])
    elastic_net_cv.fit(X_scaled, Y)
    best_alpha_enet = elastic_net_cv.alpha_
    best_l1_ratio_enet = elastic_net_cv.l1_ratio_
    print(f"\nOptimal alpha for Elastic Net: {best_alpha_enet}")
    print(f"Optimal l1_ratio for Elastic Net: {best_l1_ratio_enet}")
    
    # Compute Elastic Net path
    alphas_enet, coefs_enet, _ = enet_path(X_scaled, Y, l1_ratio=best_l1_ratio_enet, fit_intercept=True)
    plot_coefficient_convergence(alphas_enet, coefs_enet, X.columns, "Elastic Net Regression Coefficients")
    
    return {
        'LASSO': lasso_cv,
        'Ridge': ridge_cv,
        'Elastic Net': elastic_net_cv
    }

def plot_coefficient_convergence(alphas, coefs, feature_names, title):
    """Plot the convergence of coefficients for each model"""
    plt.figure(figsize=(12, 8))
    for coef, name in zip(coefs, feature_names):
        plt.plot(np.log10(alphas), coef, label=name)
    plt.xlabel('$\log_{10}(\lambda)$', fontsize=14)
    plt.ylabel('Coefficients', fontsize=14)
    plt.title(title, fontsize=16)
    plt.legend(loc='best', ncol=2)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def main():
    # Load and prepare the data
    filepath = r'C:\Users\azraj\OneDrive\Desktop\MA Economics\Fall Term\Causal ML\Project\Targetted Ads\replication_package_MS-INS-21-00828\at.dta'
    panel_data = pd.read_stata(filepath)
    panel_data = prepare_data(panel_data)
    
    # Run regularized models
    results = run_regularized_models(panel_data)
    
    return results

if __name__ == "__main__":
    main()


TypeError: Cannot setitem on a Categorical with a new category (missing), set the categories first

In [115]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
